In [2]:
import h5py
import numpy as np
import random
import tensorflow as tf
from matplotlib import pyplot as plt
import os

tf.reset_default_graph()


class Data:
    def __init__(self):
        with h5py.File("cell_data.h5", "r") as data:
            self.train_images = [data["/train_image_{}".format(i)][:] for i in range(28)]
            self.train_labels = [data["/train_label_{}".format(i)][:] for i in range(28)]
            self.test_images = [data["/test_image_{}".format(i)][:] for i in range(3)]
            self.test_labels = [data["/test_label_{}".format(i)][:] for i in range(3)]
    
        self.input_resolution = 300
        self.label_resolution = 116

        self.offset = (300 - 116) // 2

        image = np.empty([300, 300, 1])
        label = np.empty([116, 116])

    def get_train_image_list_and_label_list(self):
        n = random.randint(0, len(self.train_images) - 1)
        x = random.randint(0, (self.train_images[n].shape)[1] - self.input_resolution - 1)
        y = random.randint(0, (self.train_images[n].shape)[0] - self.input_resolution - 1)
        image = self.train_images[n][y:y + self.input_resolution, x:x + self.input_resolution, :]

        x += self.offset
        y += self.offset
        label = self.train_labels[n][y:y + self.label_resolution, x:x + self.label_resolution]
    
        return [image], [label]

    def get_test_image_list_and_label_list(self):
        coord_list = [[0,0], [0, 300], [218, 0], [218, 300]]
    
        image_list = []
        label_list = []
    
        for image_id in range(3):
            for y, x in coord_list:
                image = self.test_images[image_id][y:y + self.input_resolution, x:x + self.input_resolution, :]
                image_list.append(image)
                x += self.offset
                y += self.offset
                label = self.test_labels[image_id][y:y + self.label_resolution, x:x + self.label_resolution]
                label_list.append(label)
    

        return image_list, label_list

def weight_variable(shape,i):
    initial = tf.glorot_uniform_initializer()
    return tf.get_variable('a_{}'.format(i),initializer=initial,shape = shape)

def bias_variable(shape,i):
    initial = tf.zeros_initializer()
    return tf.get_variable('a_{}'.format(i), initializer=initial, shape=shape)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='VALID')

x = tf.placeholder(tf.float32, [None, 300, 300, 1])
y = tf.placeholder(tf.int32, [None, 116, 116])

# data class init
data = Data()
# train data init
image,label = data.get_train_image_list_and_label_list()


def network(x):
    i = 0
    # Weights for first convolution
    i += 1
    W_conv1 = weight_variable([3, 3, 1, 32],i)
    i += 1
    b_conv1 = bias_variable([32],i)
    # First convolution output size: 298x298x32
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    # Weight for second convultion
    i += 1
    W_conv2 = weight_variable([3, 3, 32, 32],i)
    i += 1
    b_conv2 = bias_variable([32],i)
    # Second convolution output size: 296x296x32
    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
    # First pooling with filter 2x2 output size: 148x148x32
    h_pool1 = max_pool_2x2(h_conv2)
    # Weight for third convolution
    i += 1
    W_conv3 = weight_variable([3, 3, 32, 64],i)
    i += 1
    b_conv3 = bias_variable([64],i)
    # Third convolution output size: 146x146x64
    h_conv3 = tf.nn.relu(conv2d(h_pool1, W_conv3) + b_conv3)
    # Weight for fourth convolution
    i += 1
    W_conv4 = weight_variable([3, 3, 64, 64],i)
    i += 1
    b_conv4 = bias_variable([64],i)
    # Fourth convolution output size 144x144x64
    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4) + b_conv4)
    # Second pooling outpuz size: 72x72x64
    h_pool2 = max_pool_2x2(h_conv4)
    # Weights for fifth convolution
    i += 1
    W_conv5 = weight_variable([3, 3, 64, 128],i)
    i += 1
    b_conv5 = bias_variable([128],i)
    # Fifth convolution output size: 70x70x128
    h_conv5 = tf.nn.relu(conv2d(h_pool2, W_conv5) + b_conv5)
    # Weights for sixth convolution
    i += 1
    W_conv6 = weight_variable([3, 3, 128, 128],i)
    i += 1
    b_conv6 = bias_variable([128],i)
    # Sixth convolution output size : 68x68x128
    h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6) + b_conv6)
    # Third pooling output size : 34x34x128
    h_pool3 = max_pool_2x2(h_conv6)
    # Weights for seventh convolution
    i += 1
    W_conv7 = weight_variable([3, 3, 128, 256],i)
    i += 1
    b_conv7 = bias_variable([256],i)
    # Seventh convolution output size: 32x32x256
    h_conv7 = tf.nn.relu(conv2d(h_pool3, W_conv7) + b_conv7)
    # Weight for eight convolution
    i += 1
    W_conv8 = weight_variable([3, 3, 256, 256],i)
    i += 1
    b_conv8 = bias_variable([256],i)
    # Eight convolution output size : 30x30x256
    h_conv8 = tf.nn.relu(conv2d(h_conv7, W_conv8) + b_conv8)
    # Forth pooling output size : 15x15x256
    h_pool4 = max_pool_2x2(h_conv8)
    # Weights for ninth convolution
    i += 1
    W_conv9 = weight_variable([3, 3, 256, 512],i)
    i += 1
    b_conv9 = bias_variable([512],i)
    # Ninth convolution output size : 13x13x512
    h_conv9 = tf.nn.relu(conv2d(h_pool4, W_conv9) + b_conv9)
    # Weights for tenth convolution
    i += 1
    W_conv10 = weight_variable([3, 3, 512, 512],i)
    i += 1
    b_conv10 = bias_variable([512],i)
    # Tenth convolution output size : 11x11x512
    h_conv10 = tf.nn.relu(conv2d(h_conv9, W_conv10) + b_conv10)
    # cropping of data
    # Croppped size: 120x120x32
    h_conv2_cropped = tf.image.resize_image_with_crop_or_pad(h_conv2,120,120)
    # Cropped size: 64x64x64
    h_conv4_cropped = tf.image.resize_image_with_crop_or_pad(h_conv4,64,64)
    # Cropped size: 36x36x128
    h_conv6_cropped = tf.image.resize_image_with_crop_or_pad(h_conv6,36,36)
    # Cropped size: 22x22x256
    h_conv8_cropped = tf.image.resize_image_with_crop_or_pad(h_conv8,22,22)
    # Output shape for deconvolution
    out_shape1 = [1, 22, 22, 256]
    # Stride size 2x2
    stride  = [1, 2, 2, 1]
    # Weight for deconvolution
    i += 1
    dw_conv1 = weight_variable([2, 2, 256, 512],i)
    # First deconvolution output shape: 22x22x256
    t_conv1 = tf.nn.conv2d_transpose(h_conv10,dw_conv1,out_shape1,stride, padding='SAME', data_format = 'NHWC')
    # First concatenation
    t_conv1_con = tf.concat([h_conv8_cropped,t_conv1],3)
    # Weightf for elevent convolution
    i += 1
    W_conv11 = weight_variable([3, 3, 512, 256],i)
    i += 1
    b_conv11 = bias_variable([256],i)
    # Eleventh convolution output shape: 20x20x256
    h_conv11 = tf.nn.relu(conv2d(t_conv1_con, W_conv11) + b_conv11)
    # Weights for twelve convolution
    i += 1
    W_conv12 = weight_variable([3, 3, 256, 256],i)
    i += 1
    b_conv12 = bias_variable([256],i)
    # Twevleth convolution output shape : 18x18x256
    h_conv12 = tf.nn.relu(conv2d(h_conv11, W_conv12) + b_conv12)
    # Output shape for second deconvolution
    out_shape2 = [1, 36, 36, 128]
    # Weights for deconvolution
    i += 1
    dw_conv2 = weight_variable([2, 2, 128, 256],i)
    # Second deconvolution output shape:36x36x128
    t_conv2 = tf.nn.conv2d_transpose(h_conv12,dw_conv2,out_shape2,stride, padding='SAME', data_format = 'NHWC')
    # Second concatenation
    t_conv2_con = tf.concat([h_conv6_cropped,t_conv2],3)
    # Weights for thirteenth convolution
    i += 1
    W_conv13 = weight_variable([3, 3, 256, 128],i)
    i += 1
    b_conv13 = bias_variable([128],i)
    # Thirteenth convolution output shape:34x34x128
    h_conv13 = tf.nn.relu(conv2d(t_conv2_con, W_conv13) + b_conv13)
    #Weights for fourteenth convolution
    i += 1
    W_conv14 = weight_variable([3, 3, 128, 128],i)
    i += 1
    b_conv14 = bias_variable([128],i)
    # Fourteenth convolution output shape: 32x32x128
    h_conv14 = tf.nn.relu(conv2d(h_conv13, W_conv14) + b_conv14)
    #Weights for deconvolution
    i += 1
    dw_conv3 = weight_variable([2, 2, 64, 128],i)
    # output shape 3
    out_shape3 = [1, 64, 64, 64]
    # Third deconvolution
    t_conv3 = tf.nn.conv2d_transpose(h_conv14,dw_conv3,out_shape3,stride, padding='SAME', data_format = 'NHWC')
    # Third concatenation
    t_conv3_con = tf.concat([h_conv4_cropped,t_conv3],3)
    # Weights for fifteenth convolution
    i += 1
    W_conv15 = weight_variable([3, 3, 128, 64],i)
    i += 1
    b_conv15 = bias_variable([64],i)
    # Fifteenth convolution output shape: 62x62x64
    h_conv15 = tf.nn.relu(conv2d(t_conv3_con, W_conv15) + b_conv15)
    # Weights for sixteents convolution
    i += 1
    W_conv16 = weight_variable([3, 3, 64, 64],i)
    i += 1
    b_conv16 = bias_variable([64],i)
    # Sixteents convolution output shape: 60x60x64
    h_conv16 = tf.nn.relu(conv2d(h_conv15, W_conv16) + b_conv16)
    # Weights for fourth deconvolution
    i += 1
    dw_conv4 = weight_variable([2, 2, 32, 64],i)
    # output shape 4
    out_shape4 = [1, 120, 120, 32]
    # Fourth deconvolution
    t_conv4 = tf.nn.conv2d_transpose(h_conv16,dw_conv4,out_shape4,stride, padding='SAME', data_format = 'NHWC')
    # Fourth concatanation
    t_conv4_con = tf.concat([h_conv2_cropped,t_conv4],3)
    # Weights for seventeenth convolution
    i += 1
    W_conv17 = weight_variable([3, 3, 64, 32],i)
    i += 1
    b_conv17 = bias_variable([32],i)
    # Seventeenth convolution output shape: 118x118x32
    h_conv17 = tf.nn.relu(conv2d(t_conv4_con, W_conv17) + b_conv17)
    # Weights for eighteenth convolution
    i += 1
    W_conv18 = weight_variable([3, 3, 32, 32],i)
    i += 1
    b_conv18 = bias_variable([32],i)
    # Eighteents convolution output shape: 116x116x32
    h_conv18 = tf.nn.relu(conv2d(h_conv17, W_conv18) + b_conv18)
    # Output convolution weights
    i += 1
    W_conv19 = weight_variable([1, 1, 32, 2],i)
    i += 1
    b_conv19 = bias_variable([2],i)
    # Output convolution with softmax outputshape 116x116x2
    h_conv19 = conv2d(h_conv18, W_conv19) + b_conv19
    return h_conv19

y_new = network(x)

loss_vector = []

# loss function
mini_batch_size = 1
learning_rate = 0.0001
y_r = tf.reshape(y, [mini_batch_size * 116 * 116])
y_h = tf.one_hot(y_r, 2)
y_new_reshape = tf.reshape(y_new,[1*116*116, 2])
loss = tf.nn.softmax_cross_entropy_with_logits(logits=y_new_reshape,labels=y_h)
loss = tf.reduce_mean(loss)
train_step = tf.train.AdamOptimizer(learning_rate, 0.95, 0.99).minimize(loss)


c_predict = tf.equal(tf.argmax(y_new_reshape, 1), tf.argmax(y_h, 1))
c_predict = tf.cast(c_predict, tf.float32)

loss_train_array = []
accuracy_train_array = []
loss_val_array = []
accuracy_val_array = []



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    loss_vector = []
    loss_vector_val = []
    for i in range(40000):
        # train process
        image, label = data.get_train_image_list_and_label_list()
        sess.run(train_step, feed_dict={x: image, y: label})
        if i % 100 == 0:
            rand = random.randint(0,12)
            # train prediction
            pred = c_predict.eval(feed_dict={x:image, y:label})
            train_acc = (np.sum(pred))/(2*pred.shape[0]-np.sum(pred))
            loss_value = loss.eval(feed_dict={x: image, y: label})
            loss_train_array.append(loss_value)
            accuracy_train_array.append(train_acc)
            # test process
            test_acc = 0
            image_list, label_list = data.get_test_image_list_and_label_list()
            for j in range(len(label_list)):

                if j == rand and i % 1000 == 0 or (j== rand and i == 39999):
                    output = y_new.eval(feed_dict={x: np.reshape(image_list[j], [1, 300, 300, 1]),
                                                   y: np.reshape(label_list[j], [1, 116, 116])})
                    output = np.argmax(output, axis=-1)
                    pic = np.reshape(output,[116,116])
                    fig1 = plt.figure(1)
                    plt.imshow(pic, cmap ='gray')
                    fig1.savefig('Prediction' + str(j)+ 'Iteration:' + str(i))
                    pic1 = np.reshape(label_list[j],[116,116])
                    fig2 = plt.figure(2)
                    plt.imshow(pic1, cmap = 'gray')
                    fig2.savefig('Label' + str(j) + 'Iteration:' + str(i))
                t_predict = c_predict.eval(feed_dict={x: np.reshape(image_list[j],[1,300,300,1]),y:np.reshape(label_list[j],[1,116,116])})
                test_acc += (np.sum(t_predict)/(2*t_predict.shape[0] - np.sum(t_predict)))
            test_acc /= len(label_list)
            print(i)
            print(test_acc)
            print(train_acc)
            accuracy_val_array.append(test_acc)
    epochs = range(len(accuracy_train_array))
    fig5 = plt.figure()
    plt.plot(epochs,accuracy_train_array)
    plt.plot(epochs, accuracy_val_array)
    plt.title('Accuracy')
    plt.legend(['Train accuracy','Validation accuracy'],loc = 'upper left')
    plt.xlabel('Epochs')
    plt.ylabel('Negative logarithm of accuracy')
    fig5.savefig('plot.jpg')
os.system('shutdown -s')



0
0.23529889469
0.240527334747
100
0.701697364725
0.979551305627
200
0.711174224266
0.953967908226
300
0.896903522924
0.9943678672
400
0.908082994378
0.923934801258
500
0.935092692002
0.933889048577
600
0.938608690526
0.950568964268
700
0.935316101806
0.858178554167
800
0.932507135336
0.961802011955
900
0.948799228026
0.965096750639
1000
0.95136800699
0.995698924731
1100
0.851934892467
0.988179669031
1200
0.953730295439
0.93974340493
1300
0.955980051104
0.986858619417
1400
0.957053803028
0.98057109214
1500
0.957009804086
1.0
1600
0.943872586307
0.849494880077
1700
0.957486010158
0.970997509887
1800
0.957665815079
0.975047702921
1900
0.955054718181
0.943385326401
2000
0.953297258123
0.9036570701
2100
0.96041926973
0.963949500109
2200
0.950035913179
0.975917767988
2300
0.965657632886
0.953400595195
2400
0.965231067512
0.978387120488
2500
0.96338615199
0.977224303872
2600
0.952572236762
0.967107667568
2700
0.965296140752
0.97867803838
2800
0.961838610318
0.963089940915
2900
0.955862737263

23600
0.968502040511
0.996883579432
23700
0.969738750041
0.98744553578
23800
0.969396710218
0.994515674794
23900
0.972698617366
0.997179962894
24000
0.970236302387
0.994811355719
24100
0.973020440104
0.989208367211
24200
0.967208995561
0.993333827124
24300
0.973371768018
0.983198231393
24400
0.970511949256
0.992595883311
24500
0.969186973879
0.994220081512
24600
0.969922966791
0.993038584018
24700
0.968911593479
0.987739124012
24800
0.973944078366
0.995402980648
24900
0.969452094259
0.99200592154
25000
0.973473973053
0.994959229059
25100
0.972086364152
0.99274342836
25200
0.972358593175
0.989649563803
25300
0.972185468415
0.99230085875
25400
0.97196671709
0.996735420686
25500
0.971997760698
0.994811355719
25600
0.97153261512
0.991858485678
25700
0.970931674889
0.991858485678
25800
0.963752564005
0.995698924731
25900
0.96968171311
0.992153379229
26000
0.968877953734
0.992890995261
26100
0.968798326933
0.991858485678
26200
0.97023362515
1.0
26300
0.966458901654
0.989208367211
26400
0.968

ValueError: Format "jpg" is not supported.
Supported formats: eps, pdf, pgf, png, ps, raw, rgba, svg, svgz.